In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np

In [2]:
def make_env():
    def make_env():
        """
        Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium.

        Questa funzione istanzia l'ambiente "Ant-v5", uno degli ambienti recenti e ben supportati
        in Gymnasium. I parametri usati sono:
        - reset_noise_scale (0.1): determina la scala del rumore quando l'ambiente viene resettato.
        - render_mode ('None'): indica che non verrà effettuato il rendering durante l'esecuzione.

        Ritorna:
            gym.Env: l'ambiente Ant-v5 inizializzato.
        """
        # Ant-v5 è l’ambiente più recente in Gymnasium.
        return gym.make("Ant-v5", reset_noise_scale=0.1, render_mode='None')
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", reset_noise_scale=0.1, render_mode='None')

In [3]:
# 1. Creiamo un ambiente vettorializzato (Vectorized Environment)
# Utilizziamo DummyVecEnv per gestire più istanze dell'ambiente come se fossero una singola entità.
# Qui passiamo la funzione make_env (definita in un'altra cella) che crea l'ambiente "Ant-v5".
env = DummyVecEnv([make_env])  

# 2. Normalizziamo osservazioni (obs) e ricompense (reward)
# VecNormalize scala le osservazioni e le ricompense per stabilizzare l'allenamento.
# Parametri:
#   norm_obs=True   -> Abilita la normalizzazione delle osservazioni.
#   norm_reward=True -> Abilita la normalizzazione delle ricompense.
#   clip_obs=10.     -> Limita i valori normalizzati dell'osservazione a un range [-10, 10] per evitare estremi.
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='None' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(


In [4]:
# 3. Definiamo il modello RL (PPO) con spiegazioni dettagliate per ciascun parametro

model = PPO(
    policy="MlpPolicy",           # Tipo di policy: una rete neurale MLP (Multilayer Perceptron) che mappa osservazioni ad azioni
    env=env,                      # Ambiente di addestramento: usa l'ambiente vettorializzato e normalizzato creato in precedenza
    learning_rate=3e-4,           # Tasso di apprendimento: controlla la velocità con cui il modello apprende aggiornando i pesi
    n_steps=2048,                 # Numero di passi da eseguire nell'ambiente per ogni ciclo di aggiornamento della policy
    batch_size=64,                # Dimensione del batch per gli aggiornamenti stocastici: suddivide i dati raccolti nei mini-batch
    n_epochs=10,                  # Numero di volte (epoch) che il dataset raccolto viene utilizzato per aggiornare la policy
    gamma=0.60,                   # Fattore di sconto: determina l'importanza delle ricompense future rispetto a quelle immediate
    gae_lambda=0.95,              # Parametro per il Generalized Advantage Estimation (GAE): bilancia bias e varianza nella stima dell'advantage
    clip_range=0.2,               # Intervallo di clipping: limita le variazioni della policy per mantenere aggiornamenti stabili
    ent_coef=0.0,                 # Coefficiente di entropia: controlla l'incentivo all'esplorazione; 0 significa nessun bonus per l'entropia
    verbose=1,                    # Livello di verbosità: 1 per stampare informazioni di log utili durante l'addestramento
    # tensorboard_log="./ppo_halfcheetah_tensorboard/"  # Opzionale: cartella per salvare i log di TensorBoard
)

Using cpu device


In [5]:
# 4. Alleniamo il modello
# Il parametro total_timesteps indica il numero totale di iterazioni (o passi)
# che il modello eseguirà durante l'allenamento. Ogni timestep rappresenta un'interazione
# con l'ambiente in cui il modello esegue un'azione e riceve un feedback, che viene poi
# usato per aggiornare la sua politica interna.
total_timesteps = 1000000  # Puoi aumentare questo valore per permettere al modello di acquisire più esperienza.
model.learn(total_timesteps=total_timesteps)

-----------------------------
| time/              |      |
|    fps             | 3268 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2608        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.020389134 |
|    clip_fraction        | 0.175       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.3       |
|    explained_variance   | -2.69       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0388     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.039      |
|    std                  | 0.995       |
|    value_loss           | 0.119       |
----------------------------------

In [6]:
#5. Salviamo il modello
model.save("ppo_Ant_model")
env.save("vecnormalize_Ant.pkl")  # salviamo anche i parametri di normalizzazione

In [7]:
# 6. Valutazione
# Qui creiamo un ambiente specifico per la valutazione del modello
#
# L'obiettivo è osservare in tempo reale come il modello interagisce con l'ambiente.
# Questo ambiente differisce da quello usato durante l'allenamento (che era vettorializzato e normalizzato)
# Per la valutazione, possiamo utilizzare l'ambiente "grezzo" di Gymnasium con rendering.
#
# Parametri:
#   - "Ant-v5": Nome dell'ambiente;
#   - reset_noise_scale=0.1: Scala del rumore applicato durante il reset dell'ambiente,
#       utile per mantenere la coerenza con le condizioni viste durante l'allenamento.
#   - render_mode="human": Abilita il rendering dell'ambiente in tempo reale, permettendoci di osservare visivamente il comportamento del modello.
eval_env = gym.make("Ant-v5", reset_noise_scale=0.1)
                    #, render_mode="human")

In [8]:
# Ricreiamo il wrapper di normalizzazione con i parametri salvati
"""
Questo snippet di codice configura un ambiente di valutazione normalizzato per un compito di reinforcement learning.
Esegue le seguenti operazioni:

1. Avvolge l'ambiente originale (eval_env) in un ambiente vettorializzato utilizzando DummyVecEnv.
    Questo è necessario per la compatibilità con algoritmi che richiedono un'interfaccia vettorializzata.
2. Carica i parametri di normalizzazione precedentemente salvati (da 'vecnormalize_Ant.pkl') nel nuovo ambiente.
    Ciò garantisce che le osservazioni dello stato (e potenzialmente le ricompense) siano normalizzate
    nello stesso modo in cui sono state normalizzate durante l'addestramento.
3. Imposta l'ambiente in modalità di valutazione:
    - Disabilita la modalità training (eval_env.training = False) in modo che le statistiche di normalizzazione non vengano aggiornate.
    - Disabilita la normalizzazione della ricompensa (eval_env.norm_reward = False) perché durante la valutazione le ricompense
      vengono tipicamente utilizzate nella loro forma originale.

Parametri:
- eval_env: L'istanza dell'ambiente che deve essere avvolta e normalizzata.
- "vecnormalize_Ant.pkl": Il percorso del file contenente i parametri di normalizzazione salvati specifici per l'ambiente 'Ant'.
  Questi parametri regolano la scala delle osservazioni e delle ricompense dell'ambiente.

Nota:
Questa configurazione è essenziale durante la valutazione di un modello addestrato per garantire che il preprocessamento
applicato corrisponda a quello dell'addestramento, senza modificare ulteriormente i parametri di normalizzazione.
"""
eval_env = DummyVecEnv([lambda: eval_env])
eval_env = VecNormalize.load("vecnormalize_Ant.pkl", eval_env)
eval_env.training = False
eval_env.norm_reward = False  # In valutazione di solito non normalizziamo la ricompensa


In [9]:
# Reset dell'ambiente di valutazione per ottenere l'osservazione iniziale.
obs = eval_env.reset()

# Inizializzazione della variabile che accumulerà la ricompensa totale ottenuta nell'episodio.
episode_reward = 0.0

# Variabile booleana per controllare se l'episodio è terminato o troncato.
done = False

# Ciclo per eseguire l'iterazione dell'episodio
while not done:
    # Otteniamo l'azione in modalità deterministica dal modello (questo assicura che il modello non agisca con esplorazione casuale)
    action, _states = model.predict(obs, deterministic=True)
    
    # Eseguiamo l'azione nell'ambiente di valutazione utilizzando il metodo step per ottenere
    # - observation: l'osservazione successiva
    # - reward: la ricompensa ottenuta per l'azione intrapresa
    # - terminated: flag che indica se l'episodio è terminato normalmente
    # - truncated: flag che indica se l'episodio è stato interrotto (es. timeout)
    observation, reward, terminated, truncated = eval_env.step(action)
    
    # Aggiornamento della ricompensa totale: poiché l'ambiente è vettorializzato, reward è un array
    episode_reward += reward[0]  
    
    # L'episodio termina se risulta terminato (terminated=True) oppure se è troncato (truncated=True)
    done = terminated or truncated

# Stampa della ricompensa totale accumulata durante l'episodio.
print("Ricompensa totale episodio:", episode_reward)

# Chiusura dell'ambiente per liberare le risorse.
eval_env.close()

Ricompensa totale episodio: 0.9071819186210632
